In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

In [ ]:
train_annotations = pd.read_csv('/content/drive/MyDrive/Oocyte.v6i.tensorflow/train/_annotations.csv')
valid_annotations = pd.read_csv('/content/drive/MyDrive/Oocyte.v6i.tensorflow/valid/_annotations.csv')
test_annotations = pd.read_csv('/content/drive/MyDrive/Oocyte.v6i.tensorflow/test/_annotations.csv')

In [ ]:
train_data_dir = '/content/drive/MyDrive/Oocyte.v6i.tensorflow/train'
valid_data_dir = '/content/drive/MyDrive/Oocyte.v6i.tensorflow/valid'
test_data_dir = '/content/drive/MyDrive/Oocyte.v6i.tensorflow/test'

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1.0/255)  # Only rescale for validation
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe=train_annotations,
                                                    directory=train_data_dir,
                                                    x_col='filename',
                                                    y_col='class',  # Change to 'class'
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

valid_generator = valid_datagen.flow_from_dataframe(dataframe=valid_annotations,
                                                    directory=valid_data_dir,
                                                    x_col='filename',
                                                    y_col='class',  # Change to 'class'
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_dataframe(dataframe=test_annotations,
                                                  directory=test_data_dir,
                                                  x_col='filename',
                                                  y_col='class',  # Change to 'class'
                                                  target_size=(img_height, img_width),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

Found 957 validated image filenames belonging to 2 classes.
Found 78 validated image filenames belonging to 2 classes.
Found 34 validated image filenames belonging to 2 classes.


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_height, img_width, 3)))


94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=valid_generator)


Epoch 1/10
30/30 [==============================] - 862s 28s/step - loss: 5.6105 - accuracy: 0.5820 - val_loss: 15953.8301 - val_accuracy: 0.5513
Epoch 2/10
30/30 [==============================] - 804s 27s/step - loss: 0.7002 - accuracy: 0.4880 - val_loss: 117.6971 - val_accuracy: 0.5513
Epoch 3/10
30/30 [==============================] - 804s 27s/step - loss: 0.6778 - accuracy: 0.5047 - val_loss: 1.2466 - val_accuracy: 0.4487
Epoch 4/10
30/30 [==============================] - 807s 27s/step - loss: 0.6448 - accuracy: 0.6280 - val_loss: 0.6944 - val_accuracy: 0.4487
Epoch 5/10
30/30 [==============================] - 801s 27s/step - loss: 0.6656 - accuracy: 0.6604 - val_loss: 0.8320 - val_accuracy: 0.4487
Epoch 6/10
30/30 [==============================] - 799s 27s/step - loss: 0.6317 - accuracy: 0.6750 - val_loss: 0.7472 - val_accuracy: 0.4487
Epoch 7/10
30/30 [==============================] - 809s 27s/step - loss: 0.5960 - accuracy: 0.7252 - val_loss: 1.1172 - val_accuracy: 0.4487


In [ ]:
train_acc = history.history['accuracy']
valid_acc = history.history['val_accuracy']

print("Training Accuracies:", train_acc)
print("Validation Accuracies:", valid_acc)

Training Accuracies: [0.5820271968841553, 0.4879832863807678, 0.5047022104263306, 0.6280041933059692, 0.6603970527648926, 0.6750261187553406, 0.7251828908920288, 0.7471264600753784, 0.7293626070022583, 0.7366771101951599]
Validation Accuracies: [0.5512820482254028, 0.5512820482254028, 0.44871795177459717, 0.44871795177459717, 0.44871795177459717, 0.44871795177459717, 0.44871795177459717, 0.44871795177459717, 0.44871795177459717, 0.44871795177459717]


In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

2/2 [==============================] - 7s 368ms/step - loss: 0.6621 - accuracy: 0.6471
Test Accuracy: 0.6470588445663452


In [ ]:
train_loss, train_acc = model.evaluate(train_generator)
print("train Accuracy:", train_acc)
valid_loss, valid_acc = model.evaluate(valid_generator)
print("valid Accuracy:", valid_acc)

30/30 [==============================] - 197s 7s/step - loss: 0.7888 - accuracy: 0.5110
train Accuracy: 0.5109717845916748
3/3 [==============================] - 14s 4s/step - loss: 0.8463 - accuracy: 0.4487
valid Accuracy: 0.44871795177459717
